In [3]:
import pandas as pd

DATA_PATH = "./data/train.csv"

raw_df = pd.read_csv(DATA_PATH)

raw_df["index"] = list(range(len(raw_df)))

train_df = raw_df.loc[:, raw_df.columns != "label"]

print(raw_df.columns, train_df.columns, len(raw_df))


'''
for dt in raw_df.dtypes :
    print(dt)
'''

Index(['WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'Dic',
       'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they',
       'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate',
       'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect',
       'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend',
       'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat',
       'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body',
       'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve',
       'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture',
       'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home',
       'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent',
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC',
       'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP',
       'label',

'\nfor dt in raw_df.dtypes :\n    print(dt)\n'

In [4]:
train_df

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,...,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,index
0,1078,51.81,51.86,38.80,55.50,25.07,21.80,74.86,43.88,14.66,...,0.37,0.19,1.48,0.00,1.95,0.0,3.62,1.67,19.20,0
1,883,12.54,52.27,60.12,88.84,19.20,14.04,92.07,60.82,20.84,...,0.34,0.00,1.70,0.23,0.91,0.0,6.12,0.45,19.71,1
2,1593,63.46,62.10,37.47,57.83,28.96,14.69,83.18,57.06,17.01,...,0.06,0.00,0.13,0.06,1.00,0.0,2.20,0.00,10.48,2
3,970,36.80,24.16,72.78,92.82,13.29,14.74,91.75,59.07,19.59,...,2.27,0.00,0.82,1.44,0.10,0.0,4.43,0.41,11.44,3
4,1339,39.78,47.91,38.15,95.89,28.49,17.63,86.41,52.73,17.48,...,5.30,0.07,1.42,1.49,1.79,0.0,1.57,0.97,12.99,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6935,1459,37.27,13.04,94.70,88.49,44.21,14.05,86.36,54.83,17.96,...,0.96,0.00,0.55,0.34,1.64,0.0,2.40,2.26,11.79,6935
6936,1654,22.17,16.36,87.19,64.73,23.63,14.51,90.57,61.91,20.19,...,0.24,0.00,0.18,0.12,0.42,0.0,6.41,0.12,9.67,6936
6937,1673,42.46,30.78,85.61,76.57,15.35,17.27,87.99,57.14,19.13,...,0.24,0.06,0.24,1.49,0.30,0.0,2.39,0.66,9.38,6937
6938,1433,25.87,55.54,30.88,95.40,11.65,15.00,88.35,55.97,17.10,...,0.21,0.07,1.05,2.58,0.70,0.0,2.37,0.98,11.30,6938


In [ ]:
import pandas as pd
import featuretools as ft
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and it includes an 'MBTI' column
# Also assuming all your LIWC results are numerical. If not, you might need to convert them first.

# Convert target variable into numerical form
le = LabelEncoder()
df['MBTI'] = le.fit_transform(df['MBTI'])

# Split data into features (X) and target (y)
X = df.drop('MBTI', axis=1)
y = df['MBTI']

# Create new entityset to hold our data
es = ft.EntitySet(id='mbti_data')

# Add the entire data frame as an entity
es = es.add_dataframe(dataframe_name='data', dataframe=X, 
                      index='index')

# Run deep feature synthesis
features, feature_defs = ft.dfs(entityset=es, target_dataframe_name='data', 
                                agg_primitives=['mean', 'max', 'min', 'std', 'skew'],
                                trans_primitives=['add_numeric', 'multiply_numeric'])

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25, random_state=42)

# Initialize TPOT
tpot = TPOTClassifier(verbosity=2, max_time_mins=10, population_size=40)

# Fit TPOT on training data
tpot.fit(X_train, y_train)

# Evaluate performance on test data
print(tpot.score(X_test, y_test))

# Export the final model
tpot.export('tpot_mbti_pipeline.py')


In [ ]:
import pandas as pd
import featuretools as ft
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split


X = train_df
y = raw_df["label"]

es = ft.EntitySet(id='mbti_data')

# Add the entire data frame as an entity
es = es.add_dataframe(
    dataframe_name='data',
    dataframe=X, 
    index='index'
)

# Run deep feature synthesis
features, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='data', 
    agg_primitives=['mean', 'max', 'min', 'std', 'skew'],
    trans_primitives=['add_numeric', 'multiply_numeric']
)

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25, random_state=42)

# Initialize TPOT
tpot = TPOTClassifier(verbosity=2, max_time_mins=10, population_size=40)

# Fit TPOT on training data
tpot.fit(X_train, y_train)

# Evaluate performance on test data
print(tpot.score(X_test, y_test))

# Export the final model
tpot.export('tpot_mbti_pipeline.py')

/opt/conda/envs/yt38/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
/opt/conda/envs/yt38/lib/python3.8/site-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['max', 'mean', 'min', 'skew', 'std']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Optimization Progress:   0%|          | 0/40 [00:00<?, ?pipeline/s]